# Instalamos e importamos librerías

In [ ]:
from builtins import round as roundpy

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=eeff0b33ed31b9fb63dd650ad7e026482dbfb9dd4361b0cd62a2deeaccb3ffa3
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-upda

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Autenticamos con Google Drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Bajamos archivo con los datos del índice BigMac

In [ ]:
#https://drive.google.com/file/d/1y4_n4twjE4VSLSC-V7QsdPJ5JR9fiVgG/view?usp=sharing GooglePlayStore.csv
#https://drive.google.com/file/d/1HBgBzv4HU1wQKKblj8p--9lJEMSGYczg/view?usp=sharing GooglePlayStore_User_Reviews.csv

id1='1y4_n4twjE4VSLSC-V7QsdPJ5JR9fiVgG'
id2='1HBgBzv4HU1wQKKblj8p--9lJEMSGYczg'
downloaded1 = drive.CreateFile({'id': id1})
downloaded1.GetContentFile('GooglePlayStore.csv')
downloaded2 = drive.CreateFile({'id': id2})
downloaded2.GetContentFile('GooglePlayStore_User_Reviews.csv')

# Creamos el Spark Context

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## Leemos CSV

In [ ]:
sqlContext = SQLContext(sc)
dfAppDetails = sqlContext.read.csv('GooglePlayStore.csv', header=True, inferSchema=True, mode='DROPMALFORMED')
dfAppReviews = sqlContext.read.csv('GooglePlayStore_User_Reviews.csv', header=True, inferSchema=True, mode='DROPMALFORMED')
rddAppDetails = dfAppDetails.rdd
rddAppReviews = dfAppReviews.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


## Preparación de Datasets

Droppeamos los apps completamente idénticos.

In [ ]:
rddAppDetails = rddAppDetails.distinct()

In [ ]:
rddAppDetails.take(5)

[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating='4.1', Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up'),
 Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating='3.9', Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up'),
 Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating='4.7', Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver='1.2.4', Android Ver='4.0.3 and up'),
 Row(App='Sketch - Draw & Paint', Category='ART_AND_DESIGN', Rating='4.5', Reviews='215644', S

# Ejercicio 15
Calcular el promedio de rating por tipo de App.

Primero filtro las aplicaciones sin valor de Rating.
Luego los mappeo a (Type, (Rating formatteado, 1)). A la hora de formatear Rating se le eliminó el punto dado que tiene formato x.x y se lo convirtió en enteros para mejorar el perfomance y reducir errores inherentes.

In [ ]:
ratingMeanPerType = rddAppDetails.filter(lambda x: x.Rating != 'NaN')
ratingMeanPerType = ratingMeanPerType.map(lambda x: (x.Type, (int(x.Rating.replace('.','')), 1)))

Mediante reduceByKey calculo para cada tipo la suma de los ratings y la cantidad de aplicaciones, para luego mappearlo a (Type, Rating promedio formatteado). Para formatear el promedio lo divido por 10 ya que le había sacado el punto y lo redondeo a 3 decimales.

In [ ]:
ratingMeanPerType = ratingMeanPerType.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
ratingMeanPerType = ratingMeanPerType.map(lambda x: (x[0], roundpy(x[1][0]/x[1][1]/10, 3)))

Utilizo take(2) sabiendo que son sólo dos tipos, para obtener los promedios para cada uno:

In [ ]:
ratingMeanPerType.take(2)

[('Free', 4.182), ('Paid', 4.262)]